In [55]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Otros objetivos
# -----------------------------------------------------------------------
import math

# Gráficos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append("../../")
from src.SupportPreProcesamiento import (
    exploracion_dataframe,
    separarar_df,
    plot_numericas,
    plot_categoricas,
    relacion_vr_categoricas,
    relacion_vr_numericas,
    matriz_correlacion,
    detectar_outliers,
    diferencia_tras_rellenar_nulos
)

pd.set_option('display.float_format', '{:.2f}'.format)


# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',100)


In [56]:
df = pd.read_csv("../../datos/api_rent_madrid.csv",sep=";")

In [57]:
df.shape

(450, 38)

# Que vamos a hacer:
- **Conocer las columnas**: Para saber que nos indican
- **Fijar la Variable Respuesta**: Ver cual es la variable a predecir
- **Eliminar duplicados:**: Antes de tocar nada, no queremos duplicados, es el único momento donde podemos hacerlo
- **Análisis exhaustivo de los datos**: Para conocer con lo que estamos trabajando
- **Fijar las Columnas categóricas**: Ver las columnas que nos otorgan precio
- **Eliminar columnas que no sean necesarias**: Para poder trabajar con los datos reducidos

In [58]:
df.sample()

,propertyCode,numPhotos,price,propertyType,operation,size,exterior,rooms,bathrooms,address,province,municipality,country,latitude,longitude,showAddress,distance,description,hasVideo,status,newDevelopment,priceByArea,detailedType,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,superTopHighlight,topNewDevelopment,externalReference,floor,district,neighborhood,hasLift,parkingSpace,labels,newDevelopmentFinished
262,92160914,28,725.00,flat,rent,67.00,True,3,1,Calle Valladolid,Madrid,Móstoles,es,403.206.268,-38.622.583,False,17191,PROMOCIÓN: 1 MES DE ALQUILER GRATIS si reserva...,True,good,False,11.00,{'typology': 'flat'},"{'subtitle': 'Centro, Móstoles', 'title': 'Pis...",True,False,True,False,False,False,CH-56672-0001,4,Centro,NaN,False,NaN,NaN,NaN


In [59]:
for col in df.columns:
    #print(f"- **{col}**:") # Imprimir columnas para markdown
    pass

 #  Conocer Columnas
- **propertyCode**: Código único de la propiedad
- **numPhotos**: número de fotos del apartamento
- **price**: Precio en euros **Variable Respuesta**
- **propertyType**: Tipo de propiedad
- **operation**: Tipo de operación (rent)
- **size**: Tamaño en m2
- **exterior**: Booleano si es exterior
- **rooms**: numero de habitacioens
- **bathrooms**: numero de baños
- **address**: Dirección aprxo de la propiedad
- **province**: Provincia
- **municipality**: Municipio
- **country**: País
- **latitude**: Latitud
- **longitude**: Longitud
- **showAddress**: Bool para mostrar dirección
- **distance**: distancia al centro en metros
- **description**: descripción
- **hasVideo**: Bool si se le saco vídeo
- **status**: 
- **newDevelopment**: Bool que indica si es nueva construcción
- **priceByArea**: Precio por m2 en €
- **detailedType**: 
- **suggestedTexts**: 
- **hasPlan**:
- **has3DTour**: Indica si tiene tour disponible
- **has360**: Indica si tiene video 360
- **hasStaging**:
- **superTopHighlight**: Bool de si estña destacada
- **topNewDevelopment**:bool de si está destacado la nueva construcción
- **externalReference**:
- **floor**: Piso de la propiedad
- **district**: Distrito
- **neighborhood**: Barrio dentro del distrito
- **hasLift**: Indica si la propiedad tiene ascensor
- **parkingSpace**: Indica info estacionamiento
- **labels**: 
- **newDevelopmentFinished**:

# Muchas columnas
- Veamos que son las que no sabemos que son

In [60]:
df["newDevelopmentFinished"].value_counts()

newDevelopmentFinished
True     12
False     5
Name: count, dtype: int64

In [61]:
df["labels"].value_counts()

labels
[{'name': 'apartamentoType', 'text': 'Apartamento'}]    30
Name: count, dtype: int64

In [62]:
df["externalReference"].value_counts().reset_index().sample()

,externalReference,count
287,40DG-000002-01,1


In [63]:
df["hasStaging"].value_counts()

hasStaging
False    432
Name: count, dtype: int64

In [64]:
df["detailedType"].value_counts()

detailedType
{'typology': 'flat'}                                           344
{'typology': 'flat', 'subTypology': 'studio'}                   46
{'typology': 'flat', 'subTypology': 'penthouse'}                16
{'typology': 'flat', 'subTypology': 'duplex'}                   15
{'typology': 'chalet', 'subTypology': 'terracedHouse'}           5
{'typology': 'chalet', 'subTypology': 'independantHouse'}        2
{'typology': 'chalet'}                                           1
{'typology': 'countryHouse', 'subTypology': 'countryHouse'}      1
{'typology': 'chalet', 'subTypology': 'semidetachedHouse'}       1
{'typology': 'countryHouse', 'subTypology': 'casaDePueblo'}      1
Name: count, dtype: int64

In [65]:
df["suggestedTexts"].value_counts()

suggestedTexts
{'subtitle': 'Suroeste, Torrejón de Ardoz', 'title': 'Piso en Calle Valle de Ambles, 2'}            10
{'subtitle': 'El Quiñón, Seseña', 'title': 'Piso'}                                                   4
{'subtitle': 'Los Rosales, Madrid', 'title': 'Piso en Calle de Sáhara'}                              3
{'subtitle': 'Las Sedas - El Olivar, Alcalá de Henares', 'title': 'Piso en Calle de Honduras'}       3
{'subtitle': 'Cabañas de la Sagra', 'title': 'Piso'}                                                 2
                                                                                                    ..
{'subtitle': 'Rascafría', 'title': 'Piso'}                                                           1
{'subtitle': 'Cedillo del Condado', 'title': 'Piso en Calle Arena'}                                  1
{'subtitle': 'Recas', 'title': 'Piso en Calle Arroyo'}                                               1
{'subtitle': 'Villamanrique de Tajo', 'title': 'Ático en C

In [66]:
df["hasPlan"].value_counts()

hasPlan
False    296
True     136
Name: count, dtype: int64

In [67]:
df["status"].value_counts()

status
good              413
newdevelopment     17
renew               2
Name: count, dtype: int64

- **newDevelopmentFinished**: Describe si se ha terminado la obra
- **labels**: Tipo apartamento, parece que siempre es "Apartamento" 
- **externalReference**: Código de referencia del piso 335 (únicos)
- **hasStaging**: No se que es, pero siempre es Falso
- **detailedType**: Tipo de vivienda detallada
- **suggestedTexts**: Título y Subtítulo Recomendados para la vivienda
- **hasPlan**: Bool, dice si la casa tiene los planos
- **status**: Estado de la casa (Bueno, nuevo, reformada)

# Ya conocemos las columnas, vamos ahora a eliminar duplicados

In [68]:
df.duplicated().sum()

np.int64(8)

In [69]:
df.drop_duplicates(inplace=True)

# Selección inicial de columnas
Vamos a seleccionar las columnas que pueden aportarnos valor al precio, son las siguientes:
- price: Es la variable Respuesta
- propertyType: Tipo Propiedad
- operation
- size: Tamaño en m2
- exterior: Bool, dice si es exterior
- rooms
- bathrooms
- address
- province
- municipality
- country
- latitude
- longitude
- distance
- status
- priceByArea
- hasPlan
- has3DTour
- has360
- superTopHighlight
- topNewDevelopment
- floor
- district
- neighborhood
- hasLift
- parkingSpace
- newDevelopmentFinished


In [70]:
df.columns

Index(['propertyCode', 'numPhotos', 'price', 'propertyType', 'operation',
       'size', 'exterior', 'rooms', 'bathrooms', 'address', 'province',
       'municipality', 'country', 'latitude', 'longitude', 'showAddress',
       'distance', 'description', 'hasVideo', 'status', 'newDevelopment',
       'priceByArea', 'detailedType', 'suggestedTexts', 'hasPlan', 'has3DTour',
       'has360', 'hasStaging', 'superTopHighlight', 'topNewDevelopment',
       'externalReference', 'floor', 'district', 'neighborhood', 'hasLift',
       'parkingSpace', 'labels', 'newDevelopmentFinished'],
      dtype='object')

In [71]:
cols = ["price","propertyType","operation","size","exterior","rooms","bathrooms",
        "address","province","municipality","country","latitude","longitude","distance",
        "status","priceByArea","hasPlan","has3DTour","has360","superTopHighlight",
        "topNewDevelopment","floor","district","neighborhood","hasLift","parkingSpace",
        "newDevelopmentFinished"]
df = df[cols]

In [72]:
df.sample()

,price,propertyType,operation,size,exterior,rooms,bathrooms,address,province,municipality,country,latitude,longitude,distance,status,priceByArea,hasPlan,has3DTour,has360,superTopHighlight,topNewDevelopment,floor,district,neighborhood,hasLift,parkingSpace,newDevelopmentFinished
332,750.00,flat,rent,42.00,False,1,1,"Paseo de las delicias, 125",Madrid,Madrid,es,403.927.535,-36.945.366,2762,good,18.00,False,False,False,False,False,4,Arganzuela,Delicias,True,NaN,NaN


# Segunda Revisión de las Columnas

In [73]:
df_cat = df.select_dtypes("O")

In [74]:
for col in df_cat.columns:
    print(f"|   Columna: {col}  |")
    print(f"No nulos: {df[col].value_counts().sum()}/{df.shape[0]}")
    print("-------------------------")
    print(f"Valores únicos:")
    print(df[col].value_counts())
    print("--------------------------------------")

|   Columna: propertyType  |
No nulos: 442/442
-------------------------
Valores únicos:
propertyType
flat            348
studio           48
duplex           19
penthouse        16
chalet            9
countryHouse      2
Name: count, dtype: int64
--------------------------------------
|   Columna: operation  |
No nulos: 442/442
-------------------------
Valores únicos:
operation
rent    442
Name: count, dtype: int64
--------------------------------------
|   Columna: address  |
No nulos: 442/442
-------------------------
Valores únicos:
address
Calle Valle de Ambles, 2             7
El Quiñón                            4
Calle de Sáhara                      3
Calle de Honduras                    3
jerez                                2
                                    ..
Calle de Ramón Esteban, 12           1
duque de medinaceli                  1
Avenida Matapiñonera, 11             1
Avenida de la Ciudad de Barcelona    1
sexta bandera                        1
Name: count, Length

# Nos quitamos columnas
No nos aportan info las siguientes
- operation: solo tiene valor "rent"
- country: solo tiene valor "es"
- ParkingSpace: Pocos datos, muchos nulos
- superTopHighlight: todo False
- topNewDevelopment: todo False
- newDevelopmentFinished: Muy pocos datos

In [76]:
cols_chao = ["operation","country","parkingSpace","superTopHighlight","topNewDevelopment","newDevelopmentFinished"]
df.drop(columns=cols_chao,inplace=True)
df.sample()


,price,propertyType,size,exterior,rooms,bathrooms,address,province,municipality,latitude,longitude,distance,status,priceByArea,hasPlan,has3DTour,has360,floor,district,neighborhood,hasLift
92,650.00,flat,65.00,True,2,1,Calle de Fuente de Lima,Madrid,Madrid,403.827.984,-3.780.637,7559,good,10.00,False,False,False,NaN,Latina,Águilas,True
